# Imports

In [6]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import io
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib

%run ../utils/object_detection_utils.ipynb
%run ../utils/data_utils.ipynb
%run ../global_variables.ipynb
%run ./detect_variables.ipynb

MODEL_ROOT = os.path.join(OD_ROOT, "ws_bd/")

In [7]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
utils_ops.tf = tf.compat.v1
tf.gfile = tf.io.gfile

# Loading model

In [8]:
# Model
model = tf.saved_model.load(os.path.join(MODEL_ROOT,"export/saved_model/"))
model = model.signatures['serving_default']
# Category index
PATH_TO_LABELS = os.path.join(MODEL_ROOT,"dataset/binary_label_map.pbtxt")
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


# Visual test

In [47]:
def draw_bbs(source, bbs, bbs_scores=None, min_score=0, colors=None):
    img = source.copy()
    for i, bb in enumerate(bbs):
        if bbs_scores is None or bbs_scores[i]>min_score:
            ymin, xmin, ymax, xmax = bb
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255, 0, 0), 4)
            if not bbs_scores is None:
                color = (255, 0, 0) if (colors is None) else colors[i]
                font = cv2.FONT_HERSHEY_SIMPLEX
                text_scale=2
                cv2.putText(img,  "{:.2f}".format(bbs_scores[i]), (xmin+5, ymin+int(text_scale*30)), font, text_scale*1, color, 3, cv2.LINE_AA)
    return img

In [49]:
annotation_root=os.path.join(DATASET_ROOT, ANNOTATION_FOLDER)
annotations_paths = return_all_files_in_folder_rec([annotation_root], ["xml"])
for i, annotation_path in enumerate(annotations_paths):
    annotation = parse_annotation(annotation_path)
    if IMAGE_FOLDER is None:
        image_path = os.path.join(DATASET_ROOT, annotation["folder"], annotation["filename"])
    else:
        image_path = os.path.join(DATASET_ROOT, IMAGE_FOLDER, annotation["filename"])
    image = load_image(image_path, expand=True)
    height, width, channels = image.shape
    # Fetching groundtruth bounding boxes
    groundtruth_bbs = []
    groundtruth_bbs_colors = []
    for obj_bb in annotation["objects"]:
        if obj_bb["name"] in ["diatom"]:
            groundtruth_bbs.append([obj_bb["ymin"], obj_bb["xmin"], obj_bb["ymax"], obj_bb["xmax"]])
    # Fetching predicted bounding boxes
    prediction_result = run_inference_for_single_image(model, image)
    predicted_bbs = []
    predicted_bbs_scores = []
    for box, score in zip(prediction_result['detection_boxes'], prediction_result['detection_scores']):
        ymin, xmin, ymax, xmax = int(box[0]*height), int(box[1]*width), int(box[2]*height), int(box[3]*width)
        predicted_bbs.append([ymin, xmin, ymax, xmax])
        predicted_bbs_scores.append(score)
    #print(groundtruth_bbs, predicted_bbs)
    #display(Image.fromarray(comp_image))
    image_prediction = draw_bbs(image, predicted_bbs, bbs_scores=predicted_bbs_scores, min_score=0.8)
    image_groundtruth = draw_bbs(image, groundtruth_bbs)
    # Creating and saving comparison image
    comp_image = np.hstack((image_prediction, image_groundtruth))
    comp_image_path = os.path.join(OUTPUT_TMP, annotation["filename"]+"_comp.png")
    if cv2.imwrite(comp_image_path, comp_image):
        print("Saving:",comp_image_path)
    else:
        print("Error while saving:",comp_image_path)

Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-8-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-3-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-11-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-9-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/TEST_FECHT_06-SampleName2-1-12.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-21-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-6-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-14-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-15-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-22-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleName2-10-7.tiff_comp.png
Saving: /mnt/balrog/pfauregi/rw/tmp/RUISSEAU-FORGES_02-SampleNa

# Test with annotation

In [15]:
annotation_root=os.path.join(DATASET_ROOT, ANNOTATION_FOLDER)
annotations_paths = return_all_files_in_folder_rec([annotation_root], ["xml"])
annotation = parse_annotation(annotations_paths[0])
print(annotation)

{'folder': 'sampling_images_20200513', 'filename': 'RUISSEAU-FORGES_02-SampleName2-8-7.tiff', 'size': {'width': 2048, 'height': 2048, 'depth': 1}, 'objects': [{'name': 'diatom_debri', 'xmin': 1382, 'ymin': 1149, 'xmax': 1610, 'ymax': 1413}, {'name': 'diatom_debri', 'xmin': 725, 'ymin': 238, 'xmax': 865, 'ymax': 377}]}


In [ ]:
image_path = os.path.join(art_dataset_root, annotation["folder"], annotation["filename"])
image = load_image(image_path, expand=True)
height, width, channels = image.shape
result = run_inference_for_single_image(model, image)
#display(Image.fromarray(image))

In [ ]:
for i in range(result["num_detections"]):
    box = result["detection_boxes"][i]
    detec_scores = result["detection_scores"][i]
    if detec_scores>0.8:
        ymin, xmin, ymax, xmax = int(box[0]*height), int(box[1]*width), int(box[2]*height), int(box[3]*width)
        thumbail = image[ymin:ymax, xmin:xmax, 0]
        #display(Image.fromarray(thumbail))

In [ ]:
from tensorflow.python.summary.summary_iterator import summary_iterator

In [ ]:
for summary in summary_iterator("/mnt/nvme-storage/pfauregi/training/obj_detection/ws_bd/eval/events.out.tfevents.1589642628.balrog"):
    print(summary)